In [ ]:
import re
import zlib
import base64
import datetime
import numpy as np
import pandas as pd
import json
import configparser
from utils import *
from sys import platform
from elasticsearch import Elasticsearch

# import torch
# from tslearn.metrics import dtw, dtw_path
# from tslearn.metrics import lcss, lcss_path
# from transformers import BertTokenizer, BertModel
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# tokenizer = BertTokenizer.from_pretrained('google/bert_uncased_L-2_H-128_A-2')
# model = BertModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
# model = model.to(device)
# device

In [ ]:
cf = configparser.ConfigParser()
cf.read('config/config.cfg')

env = 'DEVELOP'
if 'win' in platform:
    env = 'DEVELOP'
elif 'linux' in platform:
    env = 'PRODUCT'
    
class EsCtrl(object):
    def __init__(self):
        self.es_ctrl = Elasticsearch(cf['ENV_'+env]['ADDR'], ca_certs=cf['ELASTICSEARCH']['CA_CERTS'])

    def query_index_logs(self, index):
        # query = {
        #     "match": {
        #         "trace": "com_ericsson_trithread:INFO"
        #     }
        # }
        #data = self.es_ctrl.search(index=index, query=query, scroll='1s', size=10000)
        data = self.es_ctrl.search(index=index, scroll='1s', size=10000)
        sid = data['_scroll_id']
        scroll_size = len(data['hits']['hits'])
        res = []
        while scroll_size > 0:
            # Before scroll, process current batch of hits
            res.extend(data['hits']['hits'])
            data = self.es_ctrl.scroll(scroll_id=sid, scroll='1s')
            # Update the scroll ID
            sid = data['_scroll_id']
            # Get the number of results that returned in the last scroll
            scroll_size = len(data['hits']['hits'])
        return res

    def query_indices(self):
        res = []
        for key in self.es_ctrl.indices.get_alias().keys():
            if len(key) > 0:
                if '.analyzed_' in key:
                    res.append(key.replace('.analyzed_', ''))
        return res

    def is_exists(self, index):
        return self.es_ctrl.indices.exists(index=index)

    def count_index(self, index):
        return self.es_ctrl.count(index=index)['count']

    def store_index(self, index, data):
        data = deflate_and_base64_encode(json.dumps(data).encode('utf-8'))
        return self.es_ctrl.index(index=index, body={'content': data})

    def query_index(self, index):
        data = self.es_ctrl.search(index=index)
        data = json.loads(decode_base64_and_inflate(data['hits']['hits'][0]['_source']['content']))
        return data

In [ ]:
############################################ Data Clean ################################################
def package_kv(df):
    res = {}
    k_type = {}
    for i, (kv,index,timestamp) in enumerate(zip(df.kv.values, df['index'].values, df.timestamp.values)):
        for item in kv:
            if len(item[1]) > 0:
                if item[0] in res:
                    res[item[0]].append(item[1]  + [timestamp] + [str(i)] + [index]) # [value1,value2,value3,timestamp,process_index,global_index]
                else:
                    res[item[0]] = [item[1]  + [timestamp] + [str(i)] + [index]]
    for key in res.keys():
        width = max(map(len, res[key])) # get max width
        type_list = []
        for i, item in enumerate(res[key]):
            if '0x' in item[0]:
                type_list.append('register')
            elif item[0].isupper():
                type_list.append('discrete')
            else:
                type_list.append('continuous')
                
            if len(item) != width:
                tmp = [0 for _ in range(0, width)]
                tmp[-3] = item[-3]
                tmp[-2] = item[-2]
                tmp[-1] = item[-1]
                res[key][i] = tmp
        res[key] = np.array(res[key]).transpose().tolist() # matrix transposition
        k_type[key] = 'discrete' if len(set(type_list)) > 1 else list(set(type_list))[0]
    return res,k_type


def package_inverted_index_table(table, key, data):
    def clean_special_symbols(text):
        for ch in ['/','*','{','}','[',']','(',')','#','+','-','!','=',':',',','"']:
            if ch in text:
                text = text.replace(ch," ")
        return re.sub(" +", " ", text)
    for index, msg in data:
        for word in set(clean_special_symbols(msg).split(' ')):
            w = word.lower()
            if w not in table:
                table[w] = {'x': [index], 'y': [key]} # x:global index, y: yaxis num 
            else:
                table[w]['x'].append(index)
                table[w]['y'].append(key)


def clean_data(esdata):
    def clean_msg_special_symbols(text):
        for ch in ['{', '}', '[', ']', '(', ')', '"', '::']:
            if ch in text:
                text = text.replace(ch, " ")
        return re.sub(" +", " ", text)

    story = []
    for item in esdata:
        if 'msg' in item['_source']:
            tmp = clean_msg_special_symbols(item['_source']['msg'])
            if len(re.findall('process \= (.*?)$', tmp)) > 0:
                process = re.findall('process \= (.*?),', tmp)[0]
                msg = re.findall('msg \= (.*?)$', tmp)[0]
            #                 fileAndLine = re.findall('fileAndLine \= \"(.*?)\"',item['_source']['msg'])[0].split(':')[0]
            # elif len(re.findall('procname \= (.*?)$', tmp)) > 0:
            #     process = re.findall('procname \= (.*?),', tmp)[0]
            #     #                 msg = tmp.split(',')[2].replace('"','').replace('}','').replace('{','')
            #     msg = tmp
            else:
                process = 'main'
                msg = tmp
                
            msg = msg.replace('= ',':').replace(' = ',':').replace(': ',':').replace(' : ',':').replace('=',':')

            for elm in re.split('[: ]',msg):
                if elm.isupper():
                    msg = re.sub('[: ]'+elm, ':'+elm, msg)

            msg = re.sub('(:(?!-).*?[ $])', r'\1,', (msg + ' $'))
            # msg
            kv = []
            for k, v in re.findall('([A-Za-z0-9_.]+?)[ ]?[:=][ ]?(.*?)[,$]', msg):
                if len(v.strip()) > 0:
                    if (v.strip()+'xx').lower()[0:2] == '0x':
                        kv.append((k.strip()+'(r)',  [v.strip()]))
                    elif v.strip()[0].isalpha():
                        kv.append((k.strip()+'(d)', [v.strip()]))
                    else:
                        kv.append((k.strip()+'(c)', re.findall('[0-9.]+', v)))
                        
            millisecond = str(item['_source']['millisecond'])
            supply_zero = ''
            for _ in range(0, 9-len(millisecond)):
                supply_zero = supply_zero + '0'
            millisecond = supply_zero + millisecond
            story.append([item['_source']['device'], item['_source']['trace'], process,  item['_source']['logtime'][:-1] + '.' + millisecond, item['_source']['msg'], kv])
            
    story = pd.DataFrame(story, columns=['device', 'trace', 'process', 'timestamp', 'msg', 'kv']).sort_values('timestamp',ascending=True).reset_index(drop=True)

    story_line = {}
    inverted_index_table = {}
    for dev in set(story.device.values):
        data = story.loc[(story['device'] == dev), :].reset_index(drop=True)
        sub_inverted_index_table = {}
        for i, process_name in enumerate(sorted(set(data.process.values), key=list(data.process.values).index)):
            process = data.loc[(data['process'] == process_name), :].reset_index()
            process['index'] = process['index'].astype(str)
            process_start_time = process['timestamp'][0]
            process_start_count = process['index'][0]
            process_end_time = process['timestamp'][process.shape[0] - 1]
            process_end_count = process['index'][process.shape[0] - 1]
            package_inverted_index_table(sub_inverted_index_table, i, zip(process['index'].values, process.msg.values))
            msg = dict(zip(process['index'].values, [str(a) + '||' + b + '||' + c for a, b, c in
                                                     zip(process.index.values, process.timestamp.values,
                                                         process.msg.values)]))

            kv,k_type = package_kv(process)
            if dev not in story_line:
                story_line[dev] = [{'process': process_name, 'start_time': process_start_time, 'start_count': process_start_count, 'end_time': process_end_time, 'end_count': process_end_count, 'msg': msg, 'kv': kv}]
            else:
                story_line[dev].append({'process': process_name, 'start_time': process_start_time, 'start_count': process_start_count, 'end_time': process_end_time, 'end_count': process_end_count, 'msg': msg, 'kv': kv})
        inverted_index_table[dev] = sub_inverted_index_table
    return {'story_line': story_line, 'inverted_index_table': inverted_index_table}


def apply_filter_by_keywords(df):
    if (len(set(df['msg']) & set([':', '='])) > 0):
        return False
    return True


def apply_filter_digit(df):
    return re.sub('\d+', '', df['msg'])


def apply_keyword_highlight(df, keywords, color_highlight):
    tmp = [item.lower() for item in keywords]
    for item in tmp:
        if (item == 'abn:') & (item in df['msg'].lower()):
            return color_highlight
        elif len(set(df['msg'].lower().split(' ')).intersection(set(tmp))) > 0:
            return color_highlight
    return df['status']


def cal_time_difference(start, end):
    return datetime.datetime.strptime(end, "%H:%M:%S") - datetime.datetime.strptime(start, "%H:%M:%S")


############################################ XML Compression and Decompression ################################################
def decode_base64_and_inflate(b64string):
    decoded_data = base64.b64decode(b64string)
    return zlib.decompress(decoded_data , -15)


def deflate_and_base64_encode(string_val):
    zlibbed_str = zlib.compress(string_val)
    compressed_string = zlibbed_str[2:-4]
    return base64.b64encode(compressed_string).decode("utf-8")

############################################ Text ecoder ################################################
def pretrained_model_encode_msg(object1, object2):
    with torch.no_grad():
        object1_inputs = tokenizer(list(object1.msg.values), padding=True, truncation=True, return_tensors="pt").to(device)
        object1_outputs = model(**object1_inputs)

        object2_inputs = tokenizer(list(object2.msg.values), padding=True, truncation=True, return_tensors="pt").to(device)
        object2_outputs = model(**object2_inputs)
    return object1_outputs, object2_outputs

def onehot_encode_string(str1,str2):
    data = []
    data.extend(str1)
    data.extend(str2)
    values = np.array(data)

    # integer encode
    label_encoder = LabelEncoder().fit(values)
    integer_encoded = label_encoder.transform(values)

    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoder = OneHotEncoder(sparse=False).fit(integer_encoded)

    encoder1 = onehot_encoder.transform(label_encoder.transform(str1).reshape(len(str1), 1))
    encoder2 = onehot_encoder.transform(label_encoder.transform(str2).reshape(len(str2), 1))
    # inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
    # print(inverted)
    return encoder1,encoder2

############################################ Space Vectors Algorithm ################################################
def cal_lcss_path_and_score(s_y1, s_y2):
    path, score = lcss_path(s_y1, s_y2)
    return path, score

def cal_dtw_path_and_score(s_y1, s_y2):
    path, score = dtw_path(s_y1, s_y2)
    return path, score

In [ ]:
index = 'exiosuu_glte_malabar_pl_2051_telog_telog_2022-09-30-15-29-19'
es_ctrl = EsCtrl()
story_data = es_ctrl.query_index_logs(index)
data = clean_data(story_data)

In [3]:
from extract import *

path = 'exiosuu_LTE_TALAGAKOCAK_GH_2052.log'
fe = FileExtract(path,'exiosuu_LTE_TALAGAKOCAK_GH_2052.log')

In [4]:
if fe.is_extractable:
    res = fe.extract()

dict_keys(['BXP_2052_radio6626'])


In [ ]:
import json
# res[0]['origin_logs']['txlProcBranchH'][0:10]
# res[0]['kv']['txlProcBranchH']['txAtt(c)']
compressed_data = deflate_and_base64_encode(json.dumps(res[0]).encode('utf-8'))

In [5]:
from extract import *

with open(cf['ENV_'+env]['LOG_STORE_PATH'] + 'ru_lock_unlock_dpd_hw_fault_air6419_mongoose_2022_10_10', "rb") as myfile:
    S = myfile.read()
res = json.loads(gzip.decompress(S))

In [7]:
res['origin_logs'].keys()

dict_keys(['AntCalGroupMgr', 'AntCalHandler', 'AntCalLogManagerProc', 'BSAas', 'CompensationManagerAas', 'EquipCtrl', 'PaSrv0:1:2:3:4:5:6:7:8:9:10:11:12:13:14:15:16:17:18:19:20:21:22:23:24:25:26:27:28:29:30:31:32:33:34:35:36:37:38:39:40:41:42:43:', 'RxBranchCtrl0', 'RxBranchCtrl1', 'RxBranchCtrl10', 'RxBranchCtrl11', 'RxBranchCtrl12', 'RxBranchCtrl13', 'RxBranchCtrl14', 'RxBranchCtrl15', 'RxBranchCtrl16', 'RxBranchCtrl17', 'RxBranchCtrl18', 'RxBranchCtrl19', 'RxBranchCtrl2', 'RxBranchCtrl20', 'RxBranchCtrl21', 'RxBranchCtrl22', 'RxBranchCtrl23', 'RxBranchCtrl24', 'RxBranchCtrl25', 'RxBranchCtrl26', 'RxBranchCtrl27', 'RxBranchCtrl28', 'RxBranchCtrl29', 'RxBranchCtrl3', 'RxBranchCtrl30', 'RxBranchCtrl31', 'RxBranchCtrl32', 'RxBranchCtrl33', 'RxBranchCtrl34', 'RxBranchCtrl35', 'RxBranchCtrl36', 'RxBranchCtrl37', 'RxBranchCtrl38', 'RxBranchCtrl39', 'RxBranchCtrl4', 'RxBranchCtrl40', 'RxBranchCtrl41', 'RxBranchCtrl42', 'RxBranchCtrl43', 'RxBranchCtrl44', 'RxBranchCtrl45', 'RxBranchCtrl46', 

In [ ]:
test['inverted_index_table']['abn']

In [ ]:
def clean_msg_special_symbols(text):
    for ch in ['{', '}', '[', ']', '(', ')', '"', '::']:
        if ch in text:
            text = text.replace(ch, " ")
    return re.sub(" +", " ", text)
    
# msg = 'GaN Boost mode, set to boost mode Pma:-15.88[-41.54 -9.50] dB, DpdPma:-20.09[-20.58 -19.18] dB, Pmb:-15.88, TorPmb:-15.92[-50.88 -9.50] dB, avgTxPma:-18.14 dB, pmDpdIrqStat:0x00000000, pmScaleFactor: 65K'
# msg = 'txAtt:145, txAttPeak:0, dpGainLoopEnable:true, dpGainCtrlType:VVA_QPB93, torTemperature:820 (0.1C), torGainBackoff:0 (0.01dB), torGainLin:3.82825(0.01dB), torStepBit:9, cc1Ctrl1=0x00000100 , avgIMpa0:690 [mAmp]'
# msg = '[TXL_GAIN] Pma:-inf[-41.54 -9.50] dB, DpdPma:-inf[-inf -inf] dB, Pmb:-inf, TorPmb:-inf[-inf -9.50] dB, avgTxPma:-inf dB, pmDpdIrqStat:0x00000000, pmScaleFactor: 65K'
# msg = 'New event= EVENT_DEACTIVATE carrierId= 196908 fbsId= 1 fbsState= DISABLED cycleRequired= YES 110'
# msg = 'Event CARRIER_DEACTIVATE for carrierId:778'
# msg = 'Set event RX_SETUP_EVENT to time: 250[ms], from 0x13000e3'
# msg = 'New event= EVENT_SETUP carrierId= 771 fbsId= 1 fbsState= SETUP cycleRequired= NO 0'
# msg = '0-insertion for fbsId=2 event=EVENT_RELEASE '
# msg = 'DP trace: 339: 4909172 dllb_radon.c(4780) INFO:7: Status: stat{dpd=0x00808c10 pd{0=0x00808c10, 1=0x00808410} ec=0x000002ff}'
msg = 'txAtt:2500, txAttPeak:0, dpGainLoopEnable:true, dpGainCtrlType:DSA_AD_TXFE, torTemperature:670 (0.1C), torGainBackoff:0 (0.01dB), torGainLin:3.61826(0.01dB), torStepBit:10, cc0Ctrl1=0x00000100 , avgIMpa0:2090 [mAmp]'
msg = clean_msg_special_symbols(msg)
msg = msg.replace('= ',':').replace(' = ',':').replace(': ',':').replace(' : ',':').replace('=',':')

for elm in re.split('[: ]',msg):
    if elm.isupper():
        msg = re.sub('[: ]'+elm, ':'+elm, msg)

msg = re.sub('(:(?!-).*?[ $])', r'\1,', (msg + ' $'))
# msg
kv = []
for k, v in re.findall('([A-Za-z0-9_.]+?)[ ]?[:=][ ]?(.*?)[,$]', msg):
    if (v.strip()+'xx').lower()[0:2] == '0x':
        kv.append((k.strip()+'(r)',  [v.strip()]))
    elif v.strip()[0].isalpha():
        kv.append((k.strip()+'(d)', [v.strip()]))
    else:
        kv.append((k.strip()+'(c)', re.findall('[0-9.]+', v)))
kv

In [ ]:
import re

def clean_msg_special_symbols(text):
    for ch in ['{', '}', '[', ']', '(', ')', '"', '::', '\'']:
        if ch in text:
            text = text.replace(ch, " ")
    return re.sub(" +", " ", text)
# msg = 'BXP_2051: [Trace log from 2 restarts before]'
# msg = 'BXP_2050: [2022-07-14 21:47:47.132] txlProcBranchA dpdController.cc:3069 INFO:txAtt:1812, txAttPeak:0, dpGainLoopEnable:false, dpGainCtrlType:IDLE, torGainLin:2.61517, torStepBit:7, ccCtrl:0x00000100, avgIMpa0:470 [mAmp]'
# msg = 'BXP_2051: [2022-07-09 15:53:37.007706415] (+?.?????????) radio6626 com_ericsson_trithread:INFO: { cpu_id = 3 }, { process = "txlProcBranchE", fileAndLine = "dpdController.cc:2112", msg = "Power measurement, Pma:-14.35[-41.54 -9.50] dB, DpdPma:-18.25[-19.05 -17.65] dB, Pmb:-14.35, TorPmb:-14.42[-49.35 -9.50] dB, avgTxPma:-13.62 dB, pmDpdIrqStat:0x00008000, pmScaleFactor: 65K" }'
# msg = '[17:46:31.176471339] (+0.000298760) air6419_mongoose com_ericsson_trithread:INFO: { cpu_id = 3 }, { process = "txlProcBranch5", fileAndLine = "delayEstGen3Drv.cc:240", msg = "Fractional delay ok. IntegerDelay: 0x0000030c, FracDelay: 0x0000003a, dpdIrqStat: 0x04000080, dpdStat: 0x04000080, txSurveyMaxDpdAddr: 0x00000101, delEstIntFracDelta: 0xffffffff, delEstIntCorr: 0x00000000, delEstFracCorr: 0x00002000" }'
msg = '{ cpu_id = 3 }, { process = "txlProcBranch5", fileAndLine = "delayEstGen3Drv.cc:240", msg = "Fractional delay ok. IntegerDelay: 0x0000030c, FracDelay: 0x0000003a, dpdIrqStat: 0x04000080, dpdStat: 0x04000080, txSurveyMaxDpdAddr: 0x00000101, delEstIntFracDelta: 0xffffffff, delEstIntCorr: 0x00000000, delEstFracCorr: 0x00002000" }'

msg = clean_msg_special_symbols(msg)
# re.findall('(.*?): \[(.*?)\] \((.*?)\) (.*?) (.*?): (.*?)$', msg)
# re.findall('\[(.*?)\] \((.*?)\) (.*?) (.*?): (.*?)$', msg)
re.findall('process \= (.*?),.*?msg \= (.*)$', msg)

In [ ]:
msg

In [ ]:
'0x00000100'.lower()

In [ ]:
'VVA_QPb93'.isupper()